# Image Cropping Test Notebook

This notebook helps test and visualize image cropping for TEM panels.

- Auto-discovers images under `out/images` and `tem2cif/io/images`.
- Provides a typed `crop_image` helper using Pillow.
- Displays original and cropped images side-by-side.
- Saves crops to `out/crops/`.

Requirements: `pillow`, `numpy`, `matplotlib`.


In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable, List, Optional, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Paths to search for images
SEARCH_DIRS: List[Path] = [
    Path("out/images"),
    Path("tem2cif/io/images"),
]

# Ensure output dir for crops
CROPS_DIR: Path = Path("out/crops")
CROPS_DIR.mkdir(parents=True, exist_ok=True)


def discover_images(extensions: Sequence[str] = (".png", ".jpg", ".jpeg", ".tif", ".tiff")) -> List[Path]:
    """Discover images under SEARCH_DIRS.

    Args:vbnmbv 
        extensions: Allowed file extensions.

    Returns:
        List of image file paths.
    """
    found: List[Path] = []
    for base in SEARCH_DIRS:
        if not base.exists():
            continue
        for ext in extensions:
            found.extend(sorted(base.rglob(f"*{ext}")))
    return found


def load_image(path: Path) -> Image.Image:
    """Load an image with Pillow.

    Args:
        path: Path to image.

    Returns:
        PIL Image in RGB mode.
    """
    img = Image.open(path)
    return img.convert("RGB")


def crop_image(img: Image.Image, bbox: Tuple[int, int, int, int]) -> Image.Image:
    """Crop image using a bounding box.

    Args:
        img: Source PIL Image.
        bbox: (left, top, right, bottom) in pixel coordinates.

    Returns:
        Cropped PIL Image.

    Raises:
        ValueError: If bbox is invalid or outside image bounds.
    """
    left, top, right, bottom = bbox

    # Validate bbox ordering and non-negative
    if right <= left or bottom <= top:
        raise ValueError(f"Invalid bbox order: {bbox}")
    if min(left, top, right, bottom) < 0:
        raise ValueError(f"Negative bbox: {bbox}")

    w, h = img.size
    # Clamp bbox to image bounds
    clamped = (
        max(0, min(left, w)),
        max(0, min(top, h)),
        max(0, min(right, w)),
        max(0, min(bottom, h)),
    )
    if clamped != bbox:
        # If clamped, also ensure non-zero area
        l, t, r, b = clamped
        if r <= l or b <= t:
            raise ValueError(f"BBox outside image bounds after clamping: {bbox} -> {clamped}")
        bbox = clamped

    return img.crop(bbox)


def show_side_by_side(original: Image.Image, cropped: Image.Image, titles: Tuple[str, str] = ("Original", "Cropped")) -> None:
    """Display original and cropped images side-by-side.

    Args:
        original: Original image.
        cropped: Cropped image.
        titles: Titles for subplots.
    """
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(original)
    axes[0].set_title(titles[0])
    axes[0].axis("off")

    axes[1].imshow(cropped)
    axes[1].set_title(titles[1])
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()


def save_crop(img: Image.Image, source_path: Path, suffix: str = "crop") -> Path:
    """Save a cropped image next to CROPS_DIR with a descriptive name.

    Args:
        img: Cropped PIL image.
        source_path: Original image path to derive name.
        suffix: Suffix to add.

    Returns:
        Path to saved crop.
    """
    stem = source_path.stem
    out_path = CROPS_DIR / f"{stem}_{suffix}.png"
    img.save(out_path)
    return out_path


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Discover images
images = discover_images()
if not images:
    print("No images found in:", [str(p) for p in SEARCH_DIRS])
else:
    print(f"Found {len(images)} images. Showing first 5:")
    for p in images[:5]:
        print(" -", p)

# Choose one image (edit this index as needed)
idx = 0
img_path = images[idx] if images else None
img = load_image(img_path) if img_path else None
img_path


In [ ]:
# Define a bbox and run a test crop
# bbox = (left, top, right, bottom)
bbox = (50, 50, 400, 400)

if img is None:
    print("No image selected. Please add images under", [str(p) for p in SEARCH_DIRS])
else:
    cropped = crop_image(img, bbox)
    show_side_by_side(img, cropped)
    out_path = save_crop(cropped, img_path, suffix=f"{bbox[0]}_{bbox[1]}_{bbox[2]}_{bbox[3]}")
    print("Saved crop to:", out_path)
